# USG Failure Prediction - Exploratory Data Analysis

**Objective:** Understand the dataset structure, identify patterns, and guide feature engineering

**Dataset:** 2,310 ultrasound devices with 44 features

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print(f"Analysis started: {datetime.now()}")

## 1. Data Loading

In [ ]:
# Load data
DATA_PATH = '../data/raw/USG_Data_cleared.csv'

try:
    df = pd.read_csv(DATA_PATH)
    print(f"✓ Data loaded successfully: {df.shape[0]} rows, {df.shape[1]} columns")
except FileNotFoundError:
    print(f"⚠ Data file not found at {DATA_PATH}")
    print("Please ensure USG_Data_cleared.csv is in the data/raw/ directory")
    df = None

In [ ]:
# Display first rows
if df is not None:
    display(df.head())
    
    # Basic info
    print("\n" + "="*60)
    print("DATASET OVERVIEW")
    print("="*60)
    df.info()

## 2. Target Variable Analysis

In [ ]:
if df is not None and 'Warranty_Claim' in df.columns:
    # Target distribution
    target_counts = df['Warranty_Claim'].value_counts()
    target_pct = df['Warranty_Claim'].value_counts(normalize=True) * 100
    
    print("Target Variable: Warranty_Claim")
    print("-" * 40)
    for label in target_counts.index:
        print(f"{label}: {target_counts[label]} ({target_pct[label]:.2f}%)")
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar plot
    target_counts.plot(kind='bar', ax=ax1, color=['#2ecc71', '#e74c3c'])
    ax1.set_title('Warranty Claim Distribution', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Warranty Claim')
    ax1.set_ylabel('Count')
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
    
    # Pie chart
    ax2.pie(target_counts, labels=target_counts.index, autopct='%1.1f%%',
            colors=['#2ecc71', '#e74c3c'], startangle=90)
    ax2.set_title('Warranty Claim Proportion', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../reports/visualizations/target_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n⚠ Class Imbalance Detected: {target_pct['Yes']:.2f}% failure rate")

## 3. Missing Values Analysis

In [ ]:
if df is not None:
    # Calculate missing values
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing_Count': missing,
        'Missing_Percentage': missing_pct
    }).sort_values('Missing_Count', ascending=False)
    
    # Show only columns with missing values
    missing_df = missing_df[missing_df['Missing_Count'] > 0]
    
    if len(missing_df) > 0:
        print("Columns with Missing Values:")
        print("-" * 60)
        display(missing_df)
        
        # Visualization
        if len(missing_df) > 0:
            plt.figure(figsize=(12, 6))
            missing_df['Missing_Percentage'].plot(kind='barh', color='coral')
            plt.title('Missing Values by Feature', fontsize=14, fontweight='bold')
            plt.xlabel('Missing Percentage (%)')
            plt.ylabel('Feature')
            plt.tight_layout()
            plt.savefig('../reports/visualizations/missing_values.png', dpi=300, bbox_inches='tight')
            plt.show()
    else:
        print("✓ No missing values found in the dataset")

## 4. Numerical Features Analysis

In [ ]:
if df is not None:
    # Identify numerical columns
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    print(f"Found {len(numerical_cols)} numerical features")
    print("\nNumerical Features Summary:")
    display(df[numerical_cols].describe())

In [ ]:
if df is not None and len(numerical_cols) > 0:
    # Distribution plots for key numerical features
    key_features = ['Assembly_Temp_C', 'Humidity_Percent', 'Solder_Temp_C', 
                   'Solder_Time_s', 'Torque_Nm', 'Gap_mm'][:6]
    key_features = [f for f in key_features if f in numerical_cols]
    
    if len(key_features) > 0:
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        axes = axes.ravel()
        
        for idx, col in enumerate(key_features):
            if idx < len(axes):
                axes[idx].hist(df[col].dropna(), bins=50, color='skyblue', edgecolor='black')
                axes[idx].set_title(f'{col} Distribution', fontweight='bold')
                axes[idx].set_xlabel(col)
                axes[idx].set_ylabel('Frequency')
        
        plt.tight_layout()
        plt.savefig('../reports/visualizations/numerical_distributions.png', dpi=300, bbox_inches='tight')
        plt.show()

## 5. Categorical Features Analysis

In [ ]:
if df is not None:
    # Identify categorical columns
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    # Exclude target and leakage columns
    exclude_cols = ['Warranty_Claim', 'Device_UUID', 'Serial_Number', 'Claim_Type']
    categorical_cols = [c for c in categorical_cols if c not in exclude_cols]
    
    print(f"Found {len(categorical_cols)} categorical features")
    
    # Show cardinality
    if len(categorical_cols) > 0:
        cardinality = {col: df[col].nunique() for col in categorical_cols}
        cardinality_df = pd.DataFrame.from_dict(cardinality, orient='index', 
                                                columns=['Unique_Values'])
        cardinality_df = cardinality_df.sort_values('Unique_Values', ascending=False)
        
        print("\nCategorical Feature Cardinality:")
        display(cardinality_df)

## 6. Feature-Target Relationship Analysis

In [ ]:
if df is not None and 'Warranty_Claim' in df.columns and len(numerical_cols) > 0:
    # Analyze numerical features vs target
    key_features = ['Assembly_Temp_C', 'Humidity_Percent', 'Solder_Temp_C', 
                   'Solder_Time_s', 'Torque_Nm', 'Gap_mm']
    key_features = [f for f in key_features if f in numerical_cols]
    
    if len(key_features) > 0:
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        axes = axes.ravel()
        
        for idx, col in enumerate(key_features[:6]):
            if idx < len(axes):
                df.boxplot(column=col, by='Warranty_Claim', ax=axes[idx])
                axes[idx].set_title(f'{col} by Warranty Claim')
                axes[idx].set_xlabel('Warranty Claim')
                axes[idx].set_ylabel(col)
        
        plt.suptitle('')
        plt.tight_layout()
        plt.savefig('../reports/visualizations/feature_target_relationship.png', dpi=300, bbox_inches='tight')
        plt.show()

## 7. Correlation Analysis

In [ ]:
if df is not None and len(numerical_cols) > 0:
    # Calculate correlation matrix
    correlation_matrix = df[numerical_cols].corr()
    
    # Visualize correlation heatmap
    plt.figure(figsize=(14, 12))
    sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', 
                center=0, square=True, linewidths=0.5)
    plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('../reports/visualizations/correlation_heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Find highly correlated pairs
    high_corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > 0.7:
                high_corr_pairs.append((
                    correlation_matrix.columns[i],
                    correlation_matrix.columns[j],
                    correlation_matrix.iloc[i, j]
                ))
    
    if high_corr_pairs:
        print("\nHighly Correlated Feature Pairs (|r| > 0.7):")
        for feat1, feat2, corr in high_corr_pairs:
            print(f"  {feat1} <-> {feat2}: {corr:.3f}")

## 8. Supplier Analysis

In [ ]:
if df is not None and 'Warranty_Claim' in df.columns:
    # Find supplier columns
    supplier_cols = [col for col in df.columns if 'Supplier' in col]
    
    if len(supplier_cols) > 0:
        print(f"Found {len(supplier_cols)} supplier columns")
        
        # Analyze failure rate by supplier (for first supplier column)
        if len(supplier_cols) > 0:
            col = supplier_cols[0]
            supplier_failure = df.groupby(col)['Warranty_Claim'].apply(
                lambda x: (x == 'Yes').sum() / len(x) * 100
            ).sort_values(ascending=False)
            
            print(f"\nFailure Rate by {col}:")
            display(supplier_failure.head(10))
            
            # Visualization
            plt.figure(figsize=(12, 6))
            supplier_failure.head(15).plot(kind='barh', color='tomato')
            plt.title(f'Failure Rate by {col} (Top 15)', fontsize=14, fontweight='bold')
            plt.xlabel('Failure Rate (%)')
            plt.ylabel('Supplier')
            plt.tight_layout()
            plt.savefig('../reports/visualizations/supplier_failure_rate.png', dpi=300, bbox_inches='tight')
            plt.show()

## 9. Batch Analysis

In [ ]:
if df is not None and 'Batch_ID' in df.columns and 'Warranty_Claim' in df.columns:
    # Batch failure rate
    batch_failure = df.groupby('Batch_ID').agg({
        'Warranty_Claim': lambda x: (x == 'Yes').sum() / len(x) * 100,
        'Device_UUID': 'count' if 'Device_UUID' in df.columns else 'size'
    }).rename(columns={'Warranty_Claim': 'Failure_Rate', 'Device_UUID': 'Batch_Size'})
    
    batch_failure = batch_failure.sort_values('Failure_Rate', ascending=False)
    
    print("Top 10 Batches by Failure Rate:")
    display(batch_failure.head(10))
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Failure rate distribution
    ax1.hist(batch_failure['Failure_Rate'], bins=30, color='steelblue', edgecolor='black')
    ax1.set_title('Distribution of Batch Failure Rates', fontweight='bold')
    ax1.set_xlabel('Failure Rate (%)')
    ax1.set_ylabel('Number of Batches')
    
    # Batch size distribution
    if 'Batch_Size' in batch_failure.columns:
        ax2.hist(batch_failure['Batch_Size'], bins=30, color='coral', edgecolor='black')
        ax2.set_title('Distribution of Batch Sizes', fontweight='bold')
        ax2.set_xlabel('Batch Size')
        ax2.set_ylabel('Number of Batches')
    
    plt.tight_layout()
    plt.savefig('../reports/visualizations/batch_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

## 10. Key Insights Summary

In [ ]:
if df is not None:
    print("="*80)
    print("KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS")
    print("="*80)
    
    if 'Warranty_Claim' in df.columns:
        failure_rate = (df['Warranty_Claim'] == 'Yes').mean() * 100
        print(f"\n1. TARGET VARIABLE:")
        print(f"   - Failure rate: {failure_rate:.2f}%")
        print(f"   - Class imbalance: Significant (requires SMOTE/class weights)")
    
    print(f"\n2. DATA QUALITY:")
    missing_total = df.isnull().sum().sum()
    print(f"   - Total missing values: {missing_total}")
    print(f"   - Data completeness: {(1 - missing_total/(df.shape[0]*df.shape[1]))*100:.2f}%")
    
    print(f"\n3. FEATURE ENGINEERING OPPORTUNITIES:")
    print(f"   - Batch-based features (age, failure rate, size)")
    print(f"   - Interaction features (Temperature x Humidity, Torque x Gap)")
    print(f"   - Supplier failure rate encoding")
    print(f"   - Anomaly detection for environmental parameters")
    
    print(f"\n4. NEXT STEPS:")
    print(f"   - Feature engineering (Notebook 02)")
    print(f"   - Model training with XGBoost (Notebook 03)")
    print(f"   - SHAP interpretability analysis (Notebook 04)")
    
    print("\n" + "="*80)
    print(f"Analysis completed: {datetime.now()}")
    print("="*80)